In [1]:
%pip install -qU sagemaker boto3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.11.2 requires botocore<1.34.35,>=1.33.2, but you have botocore 1.34.100 which is incompatible.
anthropic-bedrock 0.4.0 requires anyio<4,>=3.5.0, but you have anyio 4.2.0 which is incompatible.


In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

import boto3

account_id = boto3.client("sts").get_caller_identity().get("Account")
region_name = boto3.session.Session().region_name

In [ ]:
suffix = "com"
if region_name.startswith('cn'):
    suffix = "com.cn"
    
model_name = None
image_uri = f"{0}.dkr.ecr.{1}.amazonaws.{suffix}/story-diffusion-inference-api:latest".format(
    account_id, region_name
)
base_name = sagemaker.utils.base_name_from_image(image_uri)

In [2]:
!touch dummy
!tar czvf model.tar.gz dummy
model_data = 's3://{0}/{1}/model.tar.gz'.format(bucket, 'story-diffusion')
!aws s3 cp model.tar.gz $model_data
!rm -f dummy model.tar.gz

dummy
upload: ./model.tar.gz to s3://sagemaker-us-west-2-631023274615/story-diffusion/model.tar.gz


In [ ]:
from sagemaker.model import Model
from sagemaker.predictor import Predictor

model = Model(
    name=model_name,
    model_data=model_data,
    role=role,
    image_uri=image_uri,
    predictor_cls=Predictor,
)

In [ ]:
instance_type = "ml.g5.4xlarge"
instance_count = 1

In [ ]:
predictor = model.deploy(
    instance_type=instance_type,
    initial_instance_count=instance_count,
    container_startup_health_check_timeout=1200,
)